In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np

# test3

In [3]:
df = pd.read_csv('dataset/tweets2.csv')

In [5]:
df = df.set_index('No')

In [7]:
df.head(1)

,atConference,day,impressions,retweets,likes,userprofileclicks,urlClicks,hashtagClicks,detailExpands,follows,mediaViews,mediaEngagements
No,,,,,,,,,,,,
1,False,2017-10-26,40,0,1,0,0,0,2,0,0,0


In [11]:
df['totalSum'] = df.iloc[:, 3:].apply(lambda x: x.sum(), axis=1)

# Q1
# 위의 전처리한 데이터를 이용하여 리-트윗 횟수(retweets)가 0이 아닌 트윗을 대상으로
# 실시간 중계 여부(atConference)에 따른 좋아요(‘likes’) 클릭 수에 평균을 출력하세요.

In [14]:
df[df['retweets'] != 0].groupby('atConference')['likes'].mean()

atConference
False    15.862069
True      2.269231
Name: likes, dtype: float64

# test4

In [16]:
df = pd.read_csv('dataset/travel_insurance2.csv')
df.head(1)

,NO,Agency,AgencyType,DistributionChannel,ProductName,Claim,Duration,Destination,NetSales,Commision,Gender,Age
0,1,CBH,Travel Agency,Offline,Comprehensive Plan,No,186,MALAYSIA,-29.0,9.57,F,81


In [17]:
df = df.replace({'Null': np.nan, 'N/A': np.nan, '': np.nan})
df.shape

(63326, 12)

## 1-1

In [19]:
df = df.drop(columns=['ProductName', 'NetSales'])

## 1-2

In [20]:
df['Gender'].isna().sum()

45107

In [21]:
df.loc[(df['Gender'].isna()) & (df['NO'] % 2 == 0), 'Gender'] = 'M'
df.loc[(df['Gender'].isna()) & (df['NO'] % 2 != 0), 'Gender'] = 'F'

In [22]:
df['Gender'].isna().sum()

0

In [23]:
basetable1 = df.copy()

## Q1.

In [26]:
basetable1[(basetable1['Age'] >= 40) &\
           (basetable1['Age'] < 50) &\
           (basetable1['Gender'] == 'F')]['Destination'].value_counts()

SINGAPORE    1352
MALAYSIA      439
THAILAND      434
INDONESIA     362
AUSTRALIA     323
             ... 
BAHRAIN         1
KENYA           1
ECUADOR         1
HUNGARY         1
COLOMBIA        1
Name: Destination, Length: 72, dtype: int64

In [27]:
'MALAYSIA'

'MALAYSIA'

# test5

In [28]:
df = pd.read_csv('dataset/Train_data.csv')
df.head(2)

,ID,join_date,sex,birth_year,branch_code,P1,P2,P3,P4,P5,...,P12,P13,P14,P15,P16,P17,P18,P19,P20,P21
0,4WKQSBB,1/2/2019,F,1987,1X1H,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,CP5S02H,1/6/2019,F,1981,UAOD,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


## 1-1

In [30]:
df = df.dropna(subset=['join_date'])

In [31]:
df['join_date'].isna().sum()

0

In [38]:
df['reg_age'] = df['join_date'].str[-4:].astype(int) - df['birth_year'] + 1

In [42]:
df['reg_age'] = pd.to_datetime(df['join_date'], format='%d/%m/%Y').dt.year\
                - df['birth_year'] + 1

## 1-2

In [48]:
p_col = ['P1', 'P2', 'P3',
         'P4', 'P5', 'P6', 'P7', 'P8', 'P9', 'P10', 'P11', 'P12', 'P13', 'P14',
         'P15', 'P16', 'P17', 'P18', 'P19', 'P20', 'P21']

In [54]:
df['total'] = df[p_col].apply(lambda x: x.sum(), axis=1)

In [57]:
df['VIP'] = np.where(df['total'] >= 4, 'VIP', 'Not VIP')

In [58]:
df['VIP'].value_counts()

Not VIP    27900
VIP         1230
Name: VIP, dtype: int64

In [60]:
df.shape

(29130, 29)

In [61]:
basetable1 = df.copy()

# Q1.

In [63]:
basetable1.groupby('branch_code')['reg_age'].mean().idxmax()

'49BM'

In [66]:
basetable1.groupby('branch_code')['reg_age'].mean()\
    .sort_values(ascending=False).index[2]

'EU3L'

# Q2.

In [80]:
basetable1.groupby(['birth_year', 'sex'])['total'].sum().idxmax()

(1983, 'M')

# Q3.

In [81]:
basetable1[basetable1['birth_year'] <= 1990][['P1', 'P12']].corr().loc['P1', 'P12'].round(2)

0.01

In [82]:
round(0.010875, 2)

0.01

# Q4.

In [84]:
tmp = basetable1[(basetable1['birth_year'] <= 1980) & \
                 (basetable1['sex'] == 'F')]
tmp.head(2)

,ID,join_date,sex,birth_year,branch_code,P1,P2,P3,P4,P5,...,P15,P16,P17,P18,P19,P20,P21,reg_age,total,VIP
7,6ZOZFNF,1/5/2017,F,1977,E5SW,0,0,0,0,0,...,0,1,0,0,0,0,0,41,2,Not VIP
28,1OBRGMX,1/1/2020,F,1966,30H5,0,0,0,0,0,...,0,1,0,0,0,0,0,55,2,Not VIP


In [86]:
pd.crosstab(index=tmp['branch_code'], columns=tmp['VIP'])\
    .sort_values(by='VIP', ascending=False)

VIP,Not VIP,VIP
branch_code,,
748L,1045,50
94KC,529,35
E5SW,677,35
30H5,515,19
UAOD,547,14
XX25,472,13
ZFER,145,13
O67J,106,6
1X1H,145,5


# Q5.

In [104]:
basetable1[pd.to_datetime(basetable1['join_date'], format='%d/%m/%Y').dt.month <= 6]\
    .groupby('sex')['reg_age'].mean()

sex
F    40.314175
M    39.233391
Name: reg_age, dtype: float64

In [100]:
basetable1[basetable1['join_date'].str.split('/').str[1].astype(int) <= 6]\
    .groupby('sex')['reg_age'].mean()

sex
F    40.314175
M    39.233391
Name: reg_age, dtype: float64

In [96]:
round(abs(40.314175 - 39.233391), 1)

1.1